In [9]:
import matplotlib, os, random
import numpy as np
import imageio.v3 as imageio
from shared.Render import collect_saved_frames, render_TSP
matplotlib.use('Agg')
np.random.seed(42)

#### Helper functions for the genetic algorithm

In [10]:
def evalueate_TSP_path(ids: list, positions: np.ndarray):
    distance = 0
    for idx in range(len(ids) - 1):
        id1, id2 = ids[idx], ids[idx + 1]
        diff_x = positions[id1][0] - positions[id2][0]
        diff_y = positions[id1][1] - positions[id2][1]
        distance += np.sqrt(diff_x**2 + diff_y**2)
    id1, id2 = ids[-1], ids[0]
    diff_x = positions[id1][0] - positions[id2][0]
    diff_y = positions[id1][1] - positions[id2][1]
    distance += np.sqrt(diff_x**2 + diff_y**2)
    return distance

In [11]:
def crossover(parent_A: list, parent_B: list):
    offspring_AB = list(parent_A[:(len(parent_A) + 1) // 2])
    selected = set(offspring_AB)
    for id in parent_B:
        if id not in selected:
            offspring_AB.append(id)
    return np.array(offspring_AB)

In [12]:
def mutate(offspring_AB: list):
    a, b = random.sample(range(len(offspring_AB)), 2)
    offspring_AB[a], offspring_AB[b] = offspring_AB[b], offspring_AB[a]
    return offspring_AB

#### The genetic algorithm for TSP problem

In [13]:
def genetic_algorithm(
        positions: np.ndarray,
        num_individuals: int = 20,
        num_iter: int = 200):
    population = [np.random.permutation(len(positions)) for _ in range(num_individuals)]
    values = [evalueate_TSP_path(individual, positions) for individual in population]
    for parent_idx in range(num_individuals):
        yield None, parent_idx, None, population[parent_idx], values[parent_idx], False
    for iter in range(num_iter):
        new_population = population.copy()
        new_values = values.copy()
        for parent_A_idx in range(num_individuals):
            parent_B_idx = random.sample(range(num_individuals), 1)[0]
            offspring_AB = crossover(population[parent_A_idx], population[parent_B_idx])
            mutated = False
            if np.random.uniform() < 0.5:
                offspring_AB = mutate(offspring_AB)
                mutated = True
            offspring_value = evalueate_TSP_path(offspring_AB, positions)
            if offspring_value < values[parent_A_idx]:
                new_population[parent_A_idx] = offspring_AB
                new_values[parent_A_idx] = offspring_value
                yield iter, parent_A_idx, parent_B_idx, offspring_AB, offspring_value, mutated
        population = new_population
        values = new_values

#### Generate random positions on 2D plane

In [14]:
num_positions = 15
positions_range = (-20, 20)
positions = np.random.uniform(positions_range[0], positions_range[1], size=(num_positions, 2))

#### Run the blind search and collect frames

In [15]:
gif_folder_path = "gifs/genetic"
os.makedirs(gif_folder_path, exist_ok=True)

In [16]:
for idx, (iter, parent_A_idx, parent_B_idx, offspring, offspring_value, mutated) in enumerate(genetic_algorithm(positions)):
    if iter is None:
        text = f"Original individual {parent_A_idx}, distance: {offspring_value:.2f}"
    else:
        mutated_text = "_mutated" if mutated else ""
        text = f"Iteation: {iter} offspring {parent_A_idx}_{parent_B_idx}{mutated_text}, distance: {offspring_value}"
    frame = render_TSP(offspring, positions, text)
    imageio.imwrite(os.path.join(gif_folder_path, f"genetic{idx:08d}.png"), frame)
collect_saved_frames(gif_folder_path, f"Genetic.gif", fps=2)

#### Visualize the genetic algorithm on the TSP problem

![Genetic](gifs/genetic/Genetic.gif)